In [129]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [131]:
from collections import namedtuple
import utlis_2nd.cusdom as custom
import super_learn_task_expr2 as expr2
import warnings

read_abso_path="/Users/liangaoming/Documents/Ai_station/neural_find_sol/wgan_2nd/complex_center_dataset/combined_data.pt"
warnings.filterwarnings("ignore")
Soft_arg_temp = namedtuple("Soft_arg_temp", ["learnable", "value"])
Gumble_temp = namedtuple("Gumble_temp", ["learnable", "value"])
Sample_info = namedtuple("Sample_info", ["type", "sample_numb"])

Sinhorn_loss_info = namedtuple("Sinhorn_loss_info", ["loss_en", "p","blur"])
Fourier_loss_info = namedtuple("Fourier_loss_info", ["loss_en", "labmbda_fourier"])
Lasso_loss_info = namedtuple("Lasso_loss_info", ["loss_en", "labmbda_lasso"])



In [123]:
# split data & get the t_steps
train_loader,valid_loader,test_loader,yaml_config =\
    custom.return_train_valid_test4loader(abso_path=read_abso_path)
# look（shape）
for batch_idx, (data, target) in enumerate(train_loader):
    print("shape:", data.shape)
    print("label for csv:",target.shape)
    break


load data & read yaml
shape: torch.Size([256, 100, 9])
label for csv: torch.Size([256])


In [124]:
soft_arg_temp = Soft_arg_temp(learnable=yaml_config["Soft_argmax_info"][0],
                              value=yaml_config["Soft_argmax_info"][1])
gumble_temp = Gumble_temp(learnable=yaml_config["Gumble_info"][0],
                          value=yaml_config["Gumble_info"][1])
sample_info = Sample_info(type=yaml_config["Sample_info"][0],
                              sample_numb=yaml_config["Sample_info"][1])

In [125]:
results_path= "/Users/liangaoming/Documents/Ai_station/neural_find_sol/wgan_2nd/expr2/expr2_results/expr2_"
expr2.config["train_nomin"]= int((yaml_config['train_size_percent']*yaml_config['all_solus_numbers'])/yaml_config["batch_size"])
expr2.config["valid_nomin"]= int((yaml_config['valid_size_percent']*yaml_config['all_solus_numbers'])/yaml_config["batch_size"])
expr2.config["test_nomin"]= int((yaml_config['test_size_percent']*yaml_config['all_solus_numbers'])/yaml_config["batch_size"])
expr2.config["train_loader"]= train_loader
expr2.config["valid_loader"]= valid_loader
expr2.config["test_loader"]= test_loader
expr2.config["data_length"]= yaml_config["data_length"]
expr2.config["device"] = "cpu"
expr2.config["hidden_act"]= "rational"
expr2.config["SI_lr"]= 1e-5
expr2.config["inference_output_act"]="Identity"
expr2.config["vari_number"]= yaml_config["vari_number"]
expr2.config["sample_vesting"]=2 #2s
#about temp
expr2.config["soft_arg_info"] = soft_arg_temp
expr2.config["gumble_info"] = gumble_temp
expr2.config["sample_info"] = sample_info

#about train step and regularization
expr2.config["Inference_num_epoch"]=1

expr2.config["lamba_fourier"]= 5
expr2.config["lamba_lasso"]= 1


In [126]:
# return test' results
result,co_actor=expr2.do_expr(
                        results_save_path=results_path,
                        folder_num=97,
                        train_config=expr2.config,
                        model_type="inference_net")

the prior knowledge is {'basis_1': 'x**0', 'basis_2': 'sin', 'basis_3': 'cos'}
output_dim 202
output_act is Identity
hi_inference_net
test_only_inference
start train_inference
loss2504.216064453125_epoch0epoch_time1.998521089553833


In [127]:
from torchsummary import summary
summary(co_actor.S_I,input_size=(200,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         LayerNorm-1                  [-1, 200]             400
            Linear-2                  [-1, 512]         102,912
          Rational-3                  [-1, 512]               0
            Linear-4                  [-1, 512]         262,656
           Dropout-5                  [-1, 512]               0
          Rational-6                  [-1, 512]               0
            Linear-7                  [-1, 512]         262,656
           Dropout-8                  [-1, 512]               0
          Rational-9                  [-1, 512]               0
           Linear-10                  [-1, 202]         103,626
         Identity-11               [-1, 202, 1]               0
Total params: 732,250
Trainable params: 732,250
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/